In [21]:
! pip install -U spacy -q

In [22]:
!python -m spacy info



============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/spacy
Platform         macOS-12.3.1-arm64-arm-64bit  
Python version   3.10.6                        
Pipelines                                      



In [23]:
!pip install spacy-transformers

In [24]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [25]:
import json
f = open('/Users/aryansingh/Downloads/Predii/spacy_with_transformers/main_spacy.json')
TRAIN_DATA = json.load(f)

In [26]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:00<00:00, 381.11it/s]


In [27]:
! python -m spacy init config config.cfg --lang en --pipeline ner ! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

Usage: python -m spacy init config [OPTIONS] OUTPUT_FILE
Try 'python -m spacy init config --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ No such option: -m                                                           │
╰──────────────────────────────────────────────────────────────────────────────╯


In [11]:
import spacy

gpu = spacy.prefer_gpu()

In [28]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.91    1.70    9.84    0.93    0.02
  6     200       5236.90  10784.84   32.73   34.54   31.10    0.33
 12     400       1567.58   4273.06   72.37   70.50   74.34    0.72
 19     600        455.06   2975.27   74.58   73.03   76.21    0.75
 25     800        520.62   2959.50   76.73   77.58   75.89    0.77
 32    1000        422.52   2639.48   75.41   79.72   71.54    0.75
 38    1200        585.84   2645.31   76.07   76.25   75.89    0.76
 44    1400       2723.11   2475.35   77.22   80.34   74.34    0.77
 51    1600        366.97   2453.

In [9]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

Usage: python -m spacy train [OPTIONS] CONFIG_PATH
Try 'python -m spacy train --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ Invalid value for 'CONFIG_PATH': Path 'config.cfg' does not exist.           │
╰──────────────────────────────────────────────────────────────────────────────╯


In [10]:
nlp_ner = spacy.load("/Users/aryansingh/Downloads/Predii/spacy_with_transformers/model-last")

In [30]:
doc = nlp_ner('''conditions can result in the bottoming out the suspension and amplification of the stress placed on the floor truss network. the additional stress can result in the fracture of welds securing the floor truss network system to the chassis frame rail and/or fracture of the floor truss network support system. the possibility exists that there could be damage to electrical wiring and/or fuel lines which could potentially lead to a fire''')

In [31]:
spacy.displacy.render(doc,style="ent",jupyter=True)

In [15]:
import streamlit as st
import spacy
from spacy import displacy

# Load the NER model
nlp_ner = spacy.load("/Users/aryansingh/Downloads/Predii/spacy_with_transformers/model-best")

# Define the Streamlit app
def main():
    # Title of the app
    st.title("Named Entity Recognition with SpaCy")

    # Text input for the user to enter the text to analyze
    text_input = st.text_area("Enter text to analyze:", value="")

    # Button to trigger the analysis
    if st.button("Analyze"):
        # Process the input text with the NER model
        doc = nlp_ner(text_input)

        # Render the entities using displacy
        html = displacy.render(doc, style="ent", page=True)

        # Display the rendered entities
        st.write(html, unsafe_allow_html=True)

# Run the Streamlit app
if __name__ == "__main__":
    main()


2024-03-02 11:33:12.455 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [6]:
import spacy
from spacy.training.example import Example

nlp = spacy.load("/Users/aryansingh/Downloads/Predii/spacy_with_transformers/model-best")
examples = []
data = [("Taj mahal is in Agra.", {"entities": [(0, 9, 'name'),
(16, 20, 'place')]})]
for text, annots in data:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annots))
print(nlp.evaluate(examples)) # This will provide overall and per entity metrics

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.0, 'ents_r': 0.0, 'ents_f': 0.0, 'ents_per_type': {'name': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'place': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'speed': 2779.92321641507}


In [11]:
!python -m spacy package "/Users/aryansingh/Downloads/Predii/spacy_with_transformers/model-last" "/Users/aryansingh/Downloads/Predii/spacy_with_transformers/output" --build wheel

⚠ Generating packages without the 'build' package is deprecated and
will not be supported in the future. To install 'build': pip install build
ℹ Building package artifacts: wheel
✔ Loaded meta.json from file
/Users/aryansingh/Downloads/Predii/spacy_with_transformers/model-last/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'en_pipeline-0.0.0'
/Users/aryansingh/Downloads/Predii/spacy_with_transformers/output/en_pipeline-0.0.0
/opt/homebrew/Caskroom/miniforge/base/bin/python: No module named build
⚠ Creating wheel with 'python -m build' failed. Falling back to
deprecated use of 'wheel' with 'python setup.py bdist_wheel'
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/en_pipeline
copying en_pipeline/__init__.py -> build/lib/en_pipeline
creating build/lib/en_pipeline/en_pipeline-0.0.0
copying en_pipeline/en_pipeline-0.0.0/tokenizer -> build/lib/en_pipeline/en_pipeline-0.0.0
copying en_pipeline/

In [12]:
!pip install spacy-huggingface-hub

  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3036646042.py, line 1)

In [15]:
a = input()
print(a)

KeyboardInterrupt: Interrupted by user

In [ ]:
https://colab.research.google.com/drive/1DonxTXOg79gP_JbbtX2W9XtBggVUulXy?usp=sharing